In [33]:
# fix axes for TRS1

from pathlib import Path
import numpy as np
from multiprocessing import Pool
import logging as log
from functools import partial
log.basicConfig(level=log.INFO)
# name_re = r"*_TRS_M00506_com3.raw"
# in_dir = Path("2020_data/dati_test1_20200723")
# out_dir = Path("2020_data/dati_test1_20200723/preprocessed")

In [35]:
def process_trs1(data):
    # fix TRS1 u ax by inverting it, due to different coordinate system between trisonica and EP
    data[:, u] = -1 * data[:, u]
    return data

In [70]:
def process_m507(data):
    # this needs to rotate the axis of M507 (old TRS2) to take into account it is mounted horizontally and it is rotated.
    data[:, u] = -1 * data[:, u]
    data[:, v] = -1 * data[:, w]
    
    data[:, [u, w]] = rotate(data, np.pi/4, [u, w])
    return data

def rotate(tens, ang, axes):
    rot_matrix = np.array([[np.cos(ang), -np.sin(ang)],
                       [np.sin(ang), np.cos(ang)]])
    return np.matmul(tens[:, axes], rot_matrix)

settings for each anemometer, the processor take in input an array where columns are u,v,w,t and returns a transformed array

In [71]:
settings = {
    'm506': {
        'usecols': (10, 12, 14, 16),
        'name_re': r"*_TRS_M00506_com3.raw",
        'processor': process_trs1
        },
    'm507': {
        'usecols': (10, 12, 14, 16),
        'name_re': r"*_TRS_M00507_com4.raw",
        'processor': process_m507  
    }
}

u, v, w = 0, 1, 2 #defing costants for accessing data
# wm = {'usecols': ()}

In [72]:
def extract(usecols, processor, f):
    # log.info(f"opening {f}")
    data = np.genfromtxt(f, usecols=usecols, invalid_raise=False)
    out_name = out_dir / f"{f.name[:-4]}.csv"
    data = processor(data)
    np.savetxt(out_name, data, header="u, v, w, t", delimiter=',', fmt='%2.2f')
    log.info(f"saved file {out_name}")

In [73]:
def main():
    print("starting processing")
    if not out_dir.is_dir(): out_dir.mkdir(parents=True, exist_ok=True)
    with Pool() as p:
        for setg in settings.values():
            proc = partial(extract, setg["usecols"], setg["processor"])
            p.map(proc, in_dir.glob(setg["name_re"]))
    # for f in in_dir.glob(name_re):
    #     extract(f)

In [74]:
main()

starting processing


INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200722-1734_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200723-0400_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200722-2100_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200722-2330_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200723-0700_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200723-0830_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200723-0930_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200723-0630_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200722-2030_TRS_M00506_com3.csv
INFO:root:saved file 2020_data/dati_test1_20200723/preprocessed/20200723-0600_TRS_M00506_com3.csv
INFO:root:saved file

Testing

In [52]:
f = Path("2020_data/dati_test1_20200723/20200722-1734_TRS_M00506_com3.raw")

In [57]:
def printme(*args, **kwargs):
    print(args, kwargs)

In [58]:
for setg in settings.values():
    
    proc = partial(printme, usecols=setg["usecols"], setg["processor"])
    proc(f)

((10, 12, 14, 16), <function process_trs1 at 0x7f0b080ece60>, PosixPath('2020_data/dati_test1_20200723/20200722-1734_TRS_M00506_com3.raw'))
((10, 12, 14, 16), <function process_m507 at 0x7f0b081287a0>, PosixPath('2020_data/dati_test1_20200723/20200722-1734_TRS_M00506_com3.raw'))


In [18]:
data

array([[ 2.93,  0.24,  0.29, 28.13],
       [ 3.16,  0.28,  0.18, 27.99],
       [ 3.12,  0.27,  0.27, 27.91],
       ...,
       [ 2.91, -0.57,  0.17, 27.64],
       [ 2.66,  0.14,  0.06, 27.73],
       [ 2.56,  0.05, -0.1 , 27.69]])